<img src="saverlife.png" width="400">

# Chat Genie Demo
### Write a question and hit enter

In [1]:
import ipywidgets as widgets
from fastai.text.all import *
import urllib.request

In [2]:
def _end_headline(x):
    if type(x) != str: return ''
    if x[-1] in ['.', '?', '!', '…', ':', '“', '”']: return x + ' '
    else: return x + '. '

def prep_text(file = 'text-1-15c.csv'):
    df = pd.read_csv(file)
    df = df[~df.text.isna()].copy()
    df = df[['headline', 'segment', 'subheading', 'text',
                               'top_tag', 'url', 'text length']].copy()
    df['joincol'] = df['subheading']
    df.loc[df.joincol=='','joincol'] = df.loc[df.joincol=='','headline']
    df.joincol = df.joincol.map(_end_headline).values
    df = df[df.headline.map(lambda x: x[:33]) != "Ask an Expert episode 2 follow-up"]
    return(df)

def breakup_text(text, max=90):
    if len(text) < max:
        print(text)
    else:
        for i in range(max-1,1,-1):
            if text[i] == ' ':
                print(text[:i])
                break
        breakup_text(text[i:].strip())

In [3]:
MODEL_URL = "https://www.dropbox.com/s/tmu82r4pd9h0ek8/chat_model.pkl?dl=1"

urllib.request.urlretrieve(MODEL_URL, "model.pkl")

learn = load_learner("model.pkl")

sdf = pd.read_pickle('segment_df.pkl')

In [121]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

def predictions(text):
    pred = learn.predict(text)
    df = pd.DataFrame(pred[2], columns = ['pct'])
    df['label_idx'] = df.index
    df = df.sort_values('pct', ascending=False).iloc[:50]
    url_df = pd.DataFrame(learn.dls.vocab[1])
    url_df['label_idx'] = url_df.index
    df = df.merge(url_df)
    df.columns = ['pct', 'idx', 'joincol']
    df = df.merge(sdf)
    df['score'] = df['pct'] / (df['text length'] + 50) * 10000
    df = df.sort_values('score', ascending=False)
    return(df)

def chat_response(text, i):
    df = predictions(text)
    print(color.BOLD + color.UNDERLINE)
    breakup_text(df.iloc[i]['headline'])
    print(color.END + color.BOLD + color.DARKCYAN)
    breakup_text(df.iloc[i]['subheading'])
    print(color.END)
    breakup_text(df.iloc[i]['text'])
    print(f"\n{color.GREEN}Read the full article on the SaverLife Blog")
    print(df.iloc[i]['url'])
    print(color.END)
    
def chat1(text): chat_response(text, 0)
def chat2(text): chat_response(text, 1)
def chat3(text): chat_response(text, 2)
def chat4(text): chat_response(text, 3)
def chat5(text): chat_response(text, 4)

In [152]:
textbox = widgets.Text(
    value='How do I save more money?',
    placeholder='Write your quetion here!',
    description='Question:',
    disabled=False,
    continuous_update=False,
    layout=widgets.Layout(width='500px')
)

slider = widgets.IntSlider(
    description='Responses', max=20, min=1, value=3, continuous_update=False)

rate1 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #1', disabled=False)

rate2 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #2', disabled=False)

rate3 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #3', disabled=False)

rate4 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #4', disabled=False)

rate5 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #5', disabled=False)

out1 = widgets.interactive_output(chat1, {'text': textbox})
out2 = widgets.interactive_output(chat2, {'text': textbox})
out3 = widgets.interactive_output(chat3, {'text': textbox})
out4 = widgets.interactive_output(chat4, {'text': textbox})
out5 = widgets.interactive_output(chat5, {'text': textbox})

out_slider = widgets.interactive_output(chat_response, {'text': textbox, 'n': slider})

In [153]:
widgets.HBox([widgets.VBox([textbox])])

### Top 5 Responses

In [154]:
widgets.HBox([widgets.VBox([out1, rate1, out2, rate2, out3, rate3, out4, rate4, out5, rate5])])